In [55]:
import pandas as pd
import numpy as np
import wrangle
import warnings
warnings.filterwarnings('ignore')

from pydataset import data
from sklearn.model_selection import train_test_split

import sklearn.linear_model
import sklearn.feature_selection
import sklearn.preprocessing

# recursive feature elimination
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# 1. Load the tips dataset.

In [7]:
tips = data('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


## 1a. Create a column named tip_percentage. 
>This should be the tip amount divided by the total bill.

In [14]:
tips['tip_percentage'] = tips.tip / tips.total_bill
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


## 1b. Create a column named price_per_person. 
>This should be the total bill divided by the party size.

In [20]:
tips['price_per_person'] = tips.total_bill / tips['size']
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,6.147500


## 1c. Before using any of the methods discussed in the lesson which features do you think would be most important for predicting:
>... the tip amount? <br>
- total_bill (higher bill, tip traditionally a set percentage of bill, higher tip)
- price_per_person (higher per person price, spends more money, leaves better (or worse) tip)

>... the tip percentage?
- price_per_person (more people may mean a higher percentage of tip goes to the bill)

## 1d. Use select k best and recursive feature elimination to select the top 2 features for predicting tip amount. 
>What are they?

In [36]:
tips.head(2)

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.446667


##### We should probably take `tip_percentage` out of our feature list because this would cause Target Leak
>Our tip <b>target feature </b>is used to calculate `tip_percentage`

In [68]:
X = tips[['total_bill', 'tip', 'size', 'price_per_person']]
y = tips.tip

In [69]:
X.head()

,total_bill,tip,size,price_per_person
1,16.99,1.01,2,8.495000
2,10.34,1.66,3,3.446667
3,21.01,3.50,3,7.003333
4,23.68,3.31,2,11.840000
5,24.59,3.61,4,6.147500


In [67]:
y.head()

1    1.01
2    1.66
3    3.50
4    3.31
5    3.61
Name: tip, dtype: float64

### SelectKBest

In [70]:
from sklearn.feature_selection import SelectKBest, f_regression

# parameters: f_regression stats test, give me 2 features
f_selector = SelectKBest(f_regression, k = 2)

# find the top 2 X's correlated with y
f_selector.fit(X, y)

# boolean mask of whether the column was selected or not
feature_mask = f_selector.get_support()

# get list of top K features
f_feature = X.iloc[:,feature_mask].columns.to_list()
f_feature

['total_bill', 'size']

### Recursive Feature Elimination

In [71]:
# initializing ml algorithm
lm = LinearRegression()

# create the RFE object, indicating the ML object (ml) and the number of features I want to end up with
rfe = RFE(lm, 2)

# fit the data using RFE
rfe.fit(X, y)

# get the mask of the columns selected
feature_mask = rfe.support_

# get the list of column names
rfe_feature = X.iloc[:,feature_mask].columns.tolist()
rfe_feature

['tip', 'size']

## 1e. Use select k best and recursive feature elimination to select the top 2 features for predicting tip_percentage. 
>What are they?

In [60]:
X_per = tips[['total_bill', 'tip', 'size', 'tip', 'price_per_person']]
y_per = tips.tip_percentage

### SelectKBest

In [61]:
from sklearn.feature_selection import SelectKBest, f_regression

# parameters: f_regression stats test, give me 2 features
f_selector = SelectKBest(f_regression, k = 2)

# find the top 2 X's correlated with y
f_selector.fit(X_per, y_per)

# boolean mask of whether the column was selected or not
feature_mask = f_selector.get_support()

# get list of top K features
f_feature = X_per.iloc[:,feature_mask].columns.to_list()
f_feature

['tip', 'tip']

### Recursive Feature Elimination

In [62]:
# initializing ml algorithm
lm = LinearRegression()

# create the RFE object, indicating the ML object (ml) and the number of features I want to end up with
rfe = RFE(lm, 2)

# fit the data using RFE
rfe.fit(X_per, y_per)

# get the mask of the columns selected
feature_mask = rfe.support_

# get the list of column names
rfe_feature = X_per.iloc[:,feature_mask].columns.tolist()
rfe_feature

['size', 'tip']

# 2. `select_kbest` function
Write a function named `select_kbest` that takes in the 
>- predictors (`X`), 
>- the target (`y`), and 
>- the number of features to select (`k`)

and returns the <b>`names of the top k selected features`</b> based on the SelectKBest class.<br> <br>
<i>Test your function with the tips dataset. You should see the same results as when you did the process manually.</i>

In [79]:
def select_kbest(X, y, k):
    '''
    Takes in a df of features, series of target feature values, and k number of features to select
    when running select k best function, returning top k selected features based on the SelectKBest function
    '''
    
#     make the object
    kbest = SelectKBest(f_regression, k = k)
    
#     fit the object
    kbest.fit(X, y)
    
#     use the object (.get_support is an array of booleans to filter the list of column names)
    return X.columns[kbest.get_support()].tolist()

In [81]:
kbest_feats = select_kbest(X, y, 2)
kbest_feats

['total_bill', 'size']

# 3. `rfe` function
>Write a function named `rfe` that takes in the `predictors`, the `target`, and the `number of features to select`. It should return the `top k features` based on the RFE class. Test your function with the tips dataset. 

You should see the same results as when you did the process manually.

In [72]:
def show_feature_rankings(X, rfe):
    '''
    Takes in a df and a fit RFE object in order to output the rank of all features
    '''
    
    var_ranks = rfe.ranking_
    var_names = X.columns.tolist()
    
    ranks = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
    ranks = ranks.sort_values(by = 'Rank', ascending = True)
    
    return ranks

In [74]:
def rfe(X, y, k):
    '''
    This function takes in the predictors, target, and k --> number of features and returns the top k features
    based on RFE class
    '''
#     initialize the ML algorithm
    lm = LinearRegression()
    
#     create the rfe object, indicating the ml object (lm) and number of features to end up with --> k
    rfe = RFE(lm, 2)
    
#     fit the data using RFE
    rfe.fit(X, y)
    
#     get the mask of the columns selected
    feature_mask = rfe.support_
    
#     get the list of the column names
    rfe_features = X.iloc[:,feature_mask].columns.tolist()
    
#     show feature rankings
    all_rankings = show_feature_rankings(X, rfe)
    
    return rfe_features, all_rankings

In [78]:
rfe(X, y, 4)

(['tip', 'size'],
                 Var  Rank
 1               tip     1
 2              size     1
 3  price_per_person     2
 0        total_bill     3)